In [3]:
import os
print(os.getcwd())
!aws iam get-user

/Users/dc/aws_mlinfra
{
    "User": {
        "UserId": "669059827483",
        "Arn": "arn:aws:iam::669059827483:root",
        "CreateDate": "2008-05-27T01:57:31Z",
        "PasswordLastUsed": "2018-06-13T07:55:44Z"
    }
}


<h6>IAM</h6>
There are different accounts, 1) the admin account who is the person paying the billing. 2) this is assigned
the default user in our configuration. An iam user dclambda has also been created and is entered uncer config. This
is a blank config file under ~.aws/config. Running aws config adds a user. aws config brings up a user prompt
asking for a secret access id and secret access token. 
<img src="images/config.png">
iam is used to create user accounts. Allows other users to gain access to your account without 
sharing passwords/tokens. REsource access granted by account owner to iam users. When running an aws command the 
default user is used; if you specify --profile you can use a different user profile. Not all commands have the profile 
switch. You can switch users by using the AWS_PROFILE environment variable. export AWS_PROFILE=admin. Look in .aws/credentials
for a listing of different users. Each user has to be individually setup before uaing the aws config command. 
<p></p>
By default an iam user cant access anything in the account. 

In [2]:
#only iam users are created. Adding user names and passwords to .aws/credentaials does not create a user object
!aws iam list-users 
#shows user name, id, and access resource node for this user. 

{
    "Users": [
        {
            "Path": "/",
            "UserName": "dclambda",
            "UserId": "AIDAJBPDFGJL2TIRFHMEY",
            "Arn": "arn:aws:iam::669059827483:user/dclambda",
            "CreateDate": "2018-05-26T19:56:53Z"
        }
    ]
}


<h6>Roles</h6>
Roles are assigned to services and are used to manage access. Roles are NOT assigned users. Trusted entities assume
roles. Roles are not the same as policies. You can add policies to roles, both user defined policies and
AWS managed policies. Roles are set with permissions and a list of users
who have access. There are 4 types of roles, service roles, another AWS account, Web Identity, SAML. 
To create a role you atach permission policies to a role w/the AWS console. How do you create a service role? 
You pick the box shown
in the image blow, AWS service and then you pick the service which in this case is lambda. 
<img src="roles1.png">
Can also use boto to create a service role with a naming convention servicename.amazonaws.com. There
is no naming convention for policies; how to attach lambda for S3? S3 can access lambda and lambda can access
S3. Have to do both ways? What is easiest, assigning admin access for both services. The boto workflow is different
from the AWS one. AWS one assigns user permission to create a service linked role first; or allow IAM access first. 


<h6>Policies</h6>
Identity baesd policies and resource based policies. identities include principal identites(root user), 
Policy simulator to tst: https://policysim.aws.amazon.com/home/index.jsp?#
<p></p>
Example of policy allowing * access to DynamoDb table books
<p></p>
{</br>
  "Version": "2012-10-17",<br/>
  "Statement": {<br/>
    "Effect": "Allow",<br/>
    "Action": "dynamodb:*",<br/>
    "Resource": "arn:aws:dynamodb:us-east-2:123456789012:table/Books"<br/>
  }<br/>
}<br/>
<p></p>
To use the above policy assign this to dclambda. Show simulator and create table books. Verify access

<p>
Managed policies vs. inline policies. There are 2 types of managed policies 1) AWS managed policies and 2)customer
managed policies. Inline policies are policies embedded into a single role/user/group. </p>

<h6>Groups</h6>
Multiple policies can be organized into groups
<img src="iamgroups.png">

In [4]:
#users have roles, policies. A service role is similar to a user which contains access permissions. Users assume
#roles to access resources. This is meant to avoid situations where users have to trade access secret
#tokens to access resources

#Policies are used to grant other accounts access to resources. For example User a wants
#to grant User b access to a S3 bucket owned by User a. 

#There are root users which is who the account belongs to and iam users which
#the root user creates. Each iam user has a set of access keys. 
!aws iam list-roles

{
    "Roles": [
        {
            "Path": "/",
            "RoleName": "test",
            "RoleId": "AROAIZYR2GUNWEAWOFAQS",
            "Arn": "arn:aws:iam::669059827483:role/test",
            "CreateDate": "2018-06-13T19:51:49Z",
            "AssumeRolePolicyDocument": {
                "Version": "2012-10-17",
                "Statement": [
                    {
                        "Effect": "Allow",
                        "Principal": {
                            "Service": "s3.amazonaws.com"
                        },
                        "Action": "sts:AssumeRole"
                    }
                ]
            },
            "Description": "Allows S3 to call AWS services on your behalf."
        }
    ]
}


In [1]:
#policy document and trust relationship pollicy document. This command is looking
#for trust policy relationship policy document. 
#use attach role policy:
!aws iam attach-role-policy --policy-arn arn:aws:s3::aws:policy/PutGetListAccess --role-name S3PutGetListRole




An error occurred (NoSuchEntity) when calling the AttachRolePolicy operation: Role not found for S3PutGetListRole


In [ ]:
#use policy trust
!aws iam create-role --role-name Test-S3Role --assume-role-policy-document file://s3a.json


In [ ]:
#S3 has policy, but S3 should have resource roles Policy.json
#{
#  "Version": "2012-10-17",
#  "Statement": {
#    "Effect": "Allow",
#    "Action": "s3:List*",
#    "Resource": [
#      "arn:aws:s3:::mybucket",
#      "arn:aws:s3:::mybucket/*"
#    ]
#  }
#}

In [ ]:
https://docs.aws.amazon.com/IAM/latest/UserGuide/reference_policies_grammar.html
Unclear what the constants are

In [17]:
!aws iam list-policies
#access resource node 
#arn:partition:service:region:namespace:relative-id
#another format
#arn:aws:lambda:region:account-id:function:function-name
#there is no region, just account id: 669059827483
#"Arn": "arn:aws:iam::669059827483:policy/service-role/AWSLambdaEdgeExecutionRole-c153646e-fe2e-4221-aeb4-b729599353a5"

{
    "Policies": [
        {
            "PolicyName": "AWSLambdaEdgeExecutionRole-c153646e-fe2e-4221-aeb4-b729599353a5",
            "PolicyId": "ANPAJQVUEARZFL7CR62IK",
            "Arn": "arn:aws:iam::669059827483:policy/service-role/AWSLambdaEdgeExecutionRole-c153646e-fe2e-4221-aeb4-b729599353a5",
            "Path": "/service-role/",
            "DefaultVersionId": "v1",
            "AttachmentCount": 1,
            "IsAttachable": true,
            "CreateDate": "2018-05-26T21:07:14Z",
            "UpdateDate": "2018-05-26T21:07:14Z"
        },
        {
            "PolicyName": "CodeStar_testhtml_Contributor",
            "PolicyId": "ANPAJGWJN7FIIHG4CZ7RG",
            "Arn": "arn:aws:iam::669059827483:policy/CodeStar_testhtml_Contributor",
            "Path": "/",
            "DefaultVersionId": "v3",
            "AttachmentCount": 0,
            "IsAttachable": true,
            "CreateDate": "2017-04-21T15:43:25Z",
            "UpdateDate": "2017-04-21T15:47:24Z"
        }

            "DefaultVersionId": "v2",
            "AttachmentCount": 0,
            "IsAttachable": true,
            "CreateDate": "2015-12-11T19:08:28Z",
            "UpdateDate": "2017-02-27T21:45:50Z"
        },
        {
            "PolicyName": "AmazonKinesisReadOnlyAccess",
            "PolicyId": "ANPAIOCMTDT5RLKZ2CAJO",
            "Arn": "arn:aws:iam::aws:policy/AmazonKinesisReadOnlyAccess",
            "Path": "/",
            "DefaultVersionId": "v1",
            "AttachmentCount": 0,
            "IsAttachable": true,
            "CreateDate": "2015-02-06T18:40:30Z",
            "UpdateDate": "2015-02-06T18:40:30Z"
        },
        {
            "PolicyName": "AWSCodeDeployFullAccess",
            "PolicyId": "ANPAIONKN3TJZUKXCHXWC",
            "Arn": "arn:aws:iam::aws:policy/AWSCodeDeployFullAccess",
            "Path": "/",
            "DefaultVersionId": "v1",
            "AttachmentCount": 0,
            "IsAttachable": true,
            "CreateDate": "2015-05-19T18

In [ ]:
#example of s3 policy. Saved as file s
#there is no bucket as test
{
  "Version": "2012-10-17",
  "Statement": [
    {
      "Effect": "Allow",
      "Action": ["s3:ListBucket"],
      "Resource": ["arn:aws:s3:::test"]
    },
    {
      "Effect": "Allow",
      "Action": [
        "s3:PutObject",
        "s3:GetObject",
        "s3:DeleteObject"
      ],
      "Resource": ["arn:aws:s3:::test/*"]
    }
  ]
}